In [ ]:
import pandas as pd
import requests
import folium

# Load node and edge data from CSV files
nodes_df = pd.read_csv('pub_nodes.csv')
edges_df = pd.read_csv('pub_edges.csv')

# Function to get the driving route using OSRM
def get_osrm_route(start_coords, end_coords):
    url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}?overview=full&geometries=geojson"
    response = requests.get(url)
    
    if response.status_code == 200:
        route_data = response.json()
        if route_data['routes']:
            route_coords = route_data['routes'][0]['geometry']['coordinates']
            return [(coord[1], coord[0]) for coord in route_coords]  # Convert to (lat, long)
    return None

# Initialize a Folium map centered on the average coordinates of the nodes
m = folium.Map(location=[nodes_df['latitude'].mean(), nodes_df['longitude'].mean()], zoom_start=13)

# Add pub markers to the map with additional information
for index, row in nodes_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"<strong>{row['pub_name']}</strong>",
        icon=folium.Icon(color='green')  # Change the marker color for better visibility
    ).add_to(m)

# Draw edges (routes between pubs) using actual driving routes
for index, row in edges_df.iterrows():
    # Get the coordinates for the start and end nodes
    start_node = nodes_df[nodes_df['pub_name'] == row['source']]
    end_node = nodes_df[nodes_df['pub_name'] == row['target']]
    
    if not start_node.empty and not end_node.empty:
        start_coords = (start_node['latitude'].values[0], start_node['longitude'].values[0])
        end_coords = (end_node['latitude'].values[0], end_node['longitude'].values[0])
        
        # Get the route using OSRM
        route_coordinates = get_osrm_route(start_coords, end_coords)
        
        if route_coordinates:
            # Create a polyline for the route
            folium.PolyLine(
                locations=route_coordinates,
                color='blue',
                weight=2.5,
                opacity=0.8,
                popup=f"Distance: {row['weight']} km"  # Add distance information to the polyline
            ).add_to(m)

# Save the map to an HTML file to display later

# Display the map in a Jupyter notebook (if applicable)
m
